In [ ]:
model = tf.keras.models.load_model('my_model_pth')


In [ ]:
from jetbot import Robot
import numpy as np
import cv2

robot = Robot()

# 예측을 위해 이미지를 전처리하고 모델에 전달
def predict(image):
    # 이미지 전처리 (이미지는 모델에 맞게 조정되어야 함)
    preprocessed_image = preprocess_image(image)
    # 모델에 이미지 전달하여 예측 수행
    predictions = model.predict(np.expand_dims(preprocessed_image, axis=0))
    return predictions

# 이미지를 전처리하는 함수 (이 부분은 모델과 입력 이미지에 따라 조정해야 함)
def preprocess_image(image):
    # 예: 이미지를 모델의 입력 형식에 맞게 조정
    # 이 예에서는 이미지를 크기 224x224로 조정하고 정규화합니다.
    image = cv2.resize(image, (224, 224))
    image = image / 255.0  # 모델에 따라 정규화가 필요할 수 있습니다.
    return image

# JetBot을 제어하는 메인 함수
def control_jetbot(change):
    image = change['new']  
    predictions = predict(image)  

    red_probability = predictions[0, CATEGORIES.index('stop')]
    orange_probability = predictions[0, CATEGORIES.index('orange')]
    gesture_probability = predictions[0, CATEGORIES.index('dislike')]
    green_probability = predictions[0, CATEGORIES.index('green')]

    
    if red_probability > 0.5: 
        robot.stop()  

    
    elif orange_probability > 0.5:
        robot.backward(0.3)  

    
    elif gesture_probability > 0.5:  
        # 고개를 옆으로 젓는 동작 수행
        perform_gesture_action()
        
    elif green_probability >0.5:
        robot.forward(0.3)
        


# 고개를 옆으로 젓는 동작을 수행하는 함수 (프로젝트에 맞게 수정)
#def perform_gesture_action():
    # 여기에 제스처 동작을 추가하세요
    #print("Performing gesture action")

# 카메라 값이 변경될 때마다 JetBot을 제어하는 콜백 함수 등록
camera.observe(control_jetbot, names='value')


In [ ]:
import torch
import torchvision.transforms as transforms
from jetbot import Robot, Camera
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg

# Load the PyTorch model
model = torch.load('sign_A/my_model.pth')
model.eval()

# Initialize the JetBot robot and camera
robot = Robot()
camera = Camera.instance()

# Create image widget for real-time display
image_widget = widgets.Image(format='jpeg', width=300, height=300)

# Define the transform to preprocess the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to control JetBot based on color prediction
def control_jetbot(change):
    image = change['new']
    
    # Preprocess the image
    image_tensor = transform(image).unsqueeze(0)
    
    # Make prediction
    with torch.no_grad():
        output = model(image_tensor)
    
    # Get the predicted color
    _, predicted_idx = torch.max(output, 1)
    predicted_color = DATASETS[0].categories[predicted_idx.item()]

    # Update the image widget with the annotated image
    annotated_image = annotate_image(image, predicted_color)
    image_widget.value = bgr8_to_jpeg(annotated_image)

    # Perform corresponding action based on the predicted color
    if predicted_color == 'red':
        robot.stop()
    elif predicted_color == 'green':
        robot.forward(0.3)  # Adjust speed as needed
    elif predicted_color == 'orange':
        robot.backward(0.3)  # Adjust speed as needed
    elif predicted_color == 'gesture':
        # Perform gesture action (e.g., tilt the head)
        perform_gesture_action()

# Function to perform gesture action
def perform_gesture_action():
    # Modify this function to perform the desired gesture action
    print("Performing gesture action")

# Function to annotate the image with the predicted color
def annotate_image(image, color):
    # Add text to the image indicating the predicted color
    cv2.putText(image, f'Color: {color}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return image

# Register the control_jetbot function as a callback for the camera value change
camera.observe(control_jetbot, names='value')

# Display the image widget
display(widgets.HBox([image_widget]))


In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18  # 사용한 모델이 resnet18이라고 가정
from jetbot import Robot, Camera
from PIL import Image

# 모델 생성 (여기서는 resnet18을 사용합니다. 실제로 사용한 모델에 맞게 수정 필요)
model = resnet18(pretrained=False)  # pretrained가 False인 경우 랜덤 초기화된 모델 생성
model.fc = torch.nn.Linear(512, len(DATASETS[0].categories))  # 출력 레이어를 예측할 클래스 수에 맞게 수정

# 모델의 상태 사전 로드
model.load_state_dict(torch.load('sign_A/my_model.pth'))
model.eval()  # 모델을 평가 모드로 설정

# Initialize the JetBot robot and camera
robot = Robot()
camera = Camera.instance()

# 나머지 코드는 이전에 제공한 예제 코드와 동일


In [ ]:
import torch
from torchvision.models import resnet18
from jetbot import Robot, Camera
from PIL import Image

# 모델 생성
model = resnet18(pretrained=False)  # 예제로 resnet18을 사용하였습니다.
model.fc = torch.nn.Linear(512, len(DATASETS[0].categories))  # 출력 레이어를 예측할 클래스 수에 맞게 수정

# 저장된 모델 파일 경로
saved_model_path = 'my_model.pth'

# 저장된 모델 가중치 불러오기
model.load_state_dict(torch.load(saved_model_path, map_location=torch.device('cpu')))  # 저장할 때 사용한 장치에 따라 map_location 설정

# 모델을 평가 모드로 설정
model.eval()

# Initialize the JetBot robot and camera
robot = Robot()
camera = Camera.instance()

# 나머지 코드는 이전에 제공한 예제 코드와 동일
